In [3]:
import pandas as pd
import collections
import etl_idesp
import dim_gestor
import etl_gestores
import etl_saresp

from util import gera_id_gestao

pd.set_option('display.max_columns', None)

In [2]:
def aponta_diferencas(lista_escolas):
    n_escolas = len(ESCOLAS_FATO)
    exemplos_diferentes = set(ESCOLAS_FATO)
    print('Escolas da tabela fato: ', n_escolas)
    for item in lista_escolas:
        diferencas_fato = ESCOLAS_FATO.difference(item[1])
        exemplos_diferentes = exemplos_diferentes.intersection(diferencas_fato)
        print('Base: ', item[0], 
              ' escolas: ', len(item[1]), 
              ' faltam do fato: ', len(diferencas_fato)/n_escolas)

    print('Nao encontrado em nenhuma outra base: ', len(exemplos_diferentes))
    if len(exemplos_diferentes) > 5:
        exemplos_diferentes = list(exemplos_diferentes)[0:5]
    else:
        exemplos_diferentes = list(exemplos_diferentes)[0:len(diferencas_fato)]
    print('Exemplos nao encontrados: ', exemplos_diferentes)


# Construir a definicao da escola igual foi feito no dos gestores

In [4]:
saresp = etl_saresp.etl()

In [5]:
saresp.head(3)

,id_gestao,id_escola,ano_gestao,nota_saresp
0,esc-10005-ano-2014,10005,2014,247.4
1,esc-10005-ano-2015,10005,2015,250.5
2,esc-10005-ano-2016,10005,2016,245.6


# Dimensao endereco

In [6]:
arquivo = 'D:\\DESAFIO_SEDUC\\dados-VERSAO_1\\escolas_enderecos\\11_escolas_enderecos_0.csv'
with open(arquivo, encoding="utf8") as file_in:
    lines = []
    for line in file_in:
        lines.append(line)
dados_lines = []
len_lines = len(lines[0].split(','))
for line in lines[1:]:
    line = line.rstrip().split(',')
    if len(line) == len_lines:  # linhas com numero incorreto de colunas foram descartadas (1)
        dados_lines.append(line)
end = pd.DataFrame(data=dados_lines)
end.columns = lines[0].split(',')
end.head(3)

,nomedep,depadm,mun,codmun,de,CD_ESCOLA,CD_DIRETORIA,CD_DIRETORIA_ESTADUAL,CD_DIRETORIA_SUPVS_PROPR,NM_COMPLETO_ESCOLA,CD_UNIDADE,DS_ENDERECO,COMPLEMENTO,NUMERO,CD_ORIGEM_UNIDADE,CD_ORIGEM_ESCOLA,CD_ORIGEM_ENDERECO,LATITUDE,LONGITUDE
0,ESTADUAL - SE,1,SAO PAULO,100,NORTE 1,36444,10101,10101,10101,GENESIO DE ALMEIDA MOURA DOUTOR,37806,DOMINGOS AREVALO,RUA,862,37806,36444,37866,-23.447,-46.6967
1,ESTADUAL - SE,1,SAO PAULO,100,LESTE 5,1582,10205,10205,10205,DOMINGOS FAUSTINO SARMIENTO,24902,21 DE ABRIL,RUA,970,24902,1582,24962,-23.5422317000605,-46.6057931815944
2,ESTADUAL - SE,1,SAO PAULO,100,LESTE 5,2173,10205,10205,10205,JOAO BORGES PROFESSOR,24901,ITAPURA,RUA,976,24901,2173,24961,-23.5473,-46.566


In [37]:
end = end.rename({'CD_ESCOLA': 'id_escola'}, axis=1)
end.id_escola = end.id_escola.astype(int)
end = end.drop_duplicates(subset='id_escola')

In [38]:
sar_end = pd.merge(saresp, end.loc[:,['CD_DIRETORIA','id_escola']], on='id_escola',how='left')

In [39]:
sar_end.CD_DIRETORIA = sar_end.CD_DIRETORIA.fillna(0)

# Cluster

In [40]:
arquivo = 'D:\\DESAFIO_SEDUC\\dados-VERSAO_1\\cluster_de_escolas_similares\\12_MMR_Clusters.csv'
cluster = pd.read_csv(arquivo, sep=';')
cluster.CD_ESCOLA = cluster.CD_ESCOLA.astype(int)
cluster = cluster.rename({'CD_ESCOLA': 'id_escola'}, axis=1)
cluster = cluster.drop_duplicates(subset='id_escola')

In [19]:
cluster.head(3)

,CD_CLUSTERS,CD_DIRETORIA,id_escola,NR_CLUSTER,NR_CICLO,CD_GRUPO_CLUSTER,DT_ANO_LETIVO
0,26655,20509,24454,1,1,205091,2019
1,26656,20509,24466,1,1,205091,2019
2,26657,20509,24491,3,1,205091,2019


In [41]:
sar_ec = pd.merge(sar_end, cluster.loc[:,['NR_CLUSTER','id_escola']], on='id_escola',how='left')

In [42]:
sar_ec.NR_CLUSTER = sar_ec.NR_CLUSTER.fillna(0)

In [43]:
sar_ec.NR_CLUSTER = sar_ec.NR_CLUSTER.astype(int)

# Dimensao classes

In [32]:
arquivo = 'dados-VERSAO_1\matriculas\/10_Escolas_Classes_Qtde_Alunos.csv'
classes = pd.read_csv(arquivo, sep=';')

In [33]:
classes = classes.rename({'COD_ESC': 'id_escola'}, axis=1)
classes.id_escola = classes.id_escola.astype(int)

In [46]:
id_gestao = []
for i in classes.index:
    id_gestao.append(gera_id_gestao(classes.loc[i,'id_escola'], classes.loc[i,'ANO']))
classes['id_gestao'] = id_gestao

In [49]:
classes.QTDE_ALUNOS = classes.QTDE_ALUNOS.astype(int)

In [50]:
classes_dados = {}
classes_dados['id_gestao'] = []
classes_dados['num_alunos'] = []
classes_dados['num_tiposclasses'] = []

for nome, dados in classes.groupby('id_gestao'):
    classes_dados['id_gestao'].append(nome)
    classes_dados['num_alunos'].append(dados.QTDE_ALUNOS.sum())
    classes_dados['num_tiposclasses'].append(len(set(dados.TIPOCLASSE_DESC)))
classes_dados = pd.DataFrame(classes_dados)    

In [55]:
classes_dados.head(3)

,id_gestao,num_alunos,num_tiposclasses
0,esc-10005-ano-2014,937,1
1,esc-10005-ano-2015,901,1
2,esc-10005-ano-2016,764,1


In [56]:
sar_ecc = pd.merge(sar_ec, classes_dados, on='id_gestao',how='left')

# CARGA HORARIA

In [65]:
arquivo = 'D:\\DESAFIO_SEDUC\\dados-VERSAO_1\\carga-horria-por-docente-All-2020-04-13_1010\\BASE_CARGA_HOR_SALA_AULA_1119.csv'
hora = pd.read_csv(arquivo, sep=';')

In [66]:
hora.head(3)

,REGIAO,NOMEDE,CD_INEP,CODESC,CATESC,TPESC,UA,NOMEESC,NOMEMUN,MODAL,NOME,RG12,DI,CPF,CARGO_C,NM_CARGOC,CODMAE,DEN_CODMAE,CICLO,MATERIA,DEN_MATERIA,TOT_AULA_LIVRE,TOT_AULA_SUBST,TOT_GERAL_AULA,TOT_AULA_LIVRE_NOTURNO,TOT_AULA_SUBST_NOTURNO,TOT_AULAS_NOTURNO,JORNADA,id_interno
0,INTERIOR,D.E.REG. BAURU,35921373,921373,1,8,19742,MARIA APARECIDA MASCHIETTO OKAZAKI PROFESSORA,BAURU,EF FINAIS + EM,276873,276873,1,276873,6409,PROFESSOR EDUCACAO BASICA II,1100,LING.PORTUGUESA,CII,1100,LING.PORTUGUESA,18,0,18,0,0,0,C,276873
1,INTERIOR,D.E.REG. BAURU,35921373,921373,1,8,19742,MARIA APARECIDA MASCHIETTO OKAZAKI PROFESSORA,BAURU,EF FINAIS + EM,276873,276873,1,276873,6409,PROFESSOR EDUCACAO BASICA II,1100,LING.PORTUGUESA,EM,1111,LIN PORT LIT,8,0,8,8,0,8,C,276873
2,GDE_SP,D.E.REG. ITAQUAQUECETUBA,35007298,7298,1,8,40880,SIMON SWITZAR PADRE,POA,EF FINAIS + EM,202218,202218,2,202218,6409,PROFESSOR EDUCACAO BASICA II,1400,L. EST. INGLES,CII,1400,L. EST. INGLES,6,0,6,0,0,0,R,202218


In [67]:
hora = hora.rename({'CODESC': 'id_escola'}, axis=1)
hora.id_escola = hora.id_escola.astype(int)

In [68]:
COLUNAS_HORAS = ['TOT_AULA_LIVRE', 'TOT_AULA_SUBST', 'TOT_GERAL_AULA',
                 'TOT_AULA_LIVRE_NOTURNO', 'TOT_AULA_SUBST_NOTURNO', 
                 'TOT_AULAS_NOTURNO']

In [69]:
for col in COLUNAS_HORAS:
    hora.loc[:,col] = hora.loc[:,col].astype(int)

In [70]:
horas_aloc = {}
horas_aloc['id_escola'] = []
horas_aloc['horas_alocadas'] = []
for nome, dados in hora.groupby('id_escola'):
    horas_aloc['id_escola'].append(nome)
    total_horas = 0
    for col in COLUNAS_HORAS:
        total_horas += dados.loc[:,col].sum()
    horas_aloc['horas_alocadas'].append(total_horas)
horas_aloc = pd.DataFrame(horas_aloc)

In [74]:
sar_ecch = pd.merge(sar_ecc, horas_aloc, on='id_escola',how='left')

In [128]:
sar_ecch.head(3)

,id_gestao,id_escola,ano_gestao,nota_saresp,CD_DIRETORIA,NR_CLUSTER,num_alunos,num_tiposclasses,horas_alocadas
0,esc-10005-ano-2014,10005,2014,247.4,10702,2,937.0,1.0,2632.0
1,esc-10005-ano-2015,10005,2015,250.5,10702,2,901.0,1.0,2632.0
2,esc-10005-ano-2016,10005,2016,245.6,10702,2,764.0,1.0,2632.0


# ATRIBUICAO DIGITAL

In [84]:
arquivo = 'D:\\DESAFIO_SEDUC\\dados-VERSAO_1\\atribuicao_sem_docentes\\atribuicao_digital.csv'
atrib = pd.read_csv(arquivo, sep=',')

In [89]:
atrib = atrib.rename({'CD_ESCOLA': 'id_escola'}, axis=1)
atrib.id_escola = atrib.id_escola.astype(int)
atrib.NR_HORA_AULA_SEMANA = atrib.NR_HORA_AULA_SEMANA.astype(int)

In [90]:
atrib_dados = {}
atrib_dados['id_escola'] = []
atrib_dados['horassemana_sem_aula'] = []
for nome, dados in atrib.groupby('id_escola'):
    atrib_dados['id_escola'].append(nome)
    atrib_dados['horassemana_sem_aula'].append(dados.NR_HORA_AULA_SEMANA.sum())
atrib_dados = pd.DataFrame(atrib_dados)

In [92]:
sar_eccha = pd.merge(sar_ecch, atrib_dados, on='id_escola',how='left')

In [130]:
sar_eccha.head(3)

,id_gestao,id_escola,ano_gestao,nota_saresp,CD_DIRETORIA,NR_CLUSTER,num_alunos,num_tiposclasses,horas_alocadas,horassemana_sem_aula
0,esc-10005-ano-2014,10005,2014,247.4,10702,2,937.0,1.0,2632.0,1582.0
1,esc-10005-ano-2015,10005,2015,250.5,10702,2,901.0,1.0,2632.0,1582.0
2,esc-10005-ano-2016,10005,2016,245.6,10702,2,764.0,1.0,2632.0,1582.0


# NOTA ACUMULADA

In [107]:
dados_escola = {}
dados_escola['id_gestao'] = []
dados_escola['nota_acumulada'] = []

for nome, dados in sar_eccha.groupby('id_escola'):
    lista_anos = dados.ano_gestao.tolist()
    lista_anos = list(set(lista_anos))
    lista_anos.sort()
    nota_acumulada = 0
    accum = 0
    for ano in lista_anos:
        dados_ano = dados[dados.ano_gestao == ano]
        nota_ano = dados_ano.nota_saresp.iloc[0]
        dados_escola['id_gestao'].append(dados_ano.id_gestao.iloc[0])
        dados_escola['nota_acumulada'].append(nota_acumulada / (1 if accum == 0 else accum))
        nota_acumulada += nota_ano
        accum += 1    
dados_escola = pd.DataFrame(dados_escola)

In [109]:
sar_ecchan = pd.merge(sar_eccha, dados_escola, on='id_gestao',how='left')

In [115]:
teste_corr = sar_ecchan.drop(sar_ecchan[sar_ecchan.nota_acumulada == 0].index)

In [126]:
print('CORRELACAO EVOLUCAO: ',teste_corr.corr().loc['nota_acumulada', 'nota_saresp'])

CORRELACAO EVOLUCAO:  0.8755058495756967


# ESTATISTICAS

In [139]:
sar_ecchan.isnull().sum()

id_gestao               0
id_escola               0
ano_gestao              0
nota_saresp             0
CD_DIRETORIA            0
NR_CLUSTER              0
num_alunos              0
num_tiposclasses        0
horas_alocadas          0
horassemana_sem_aula    0
nota_acumulada          0
dtype: int64

In [138]:
sar_ecchan.num_alunos = sar_ecchan.num_alunos.fillna(sar_ecchan.num_alunos.median())
sar_ecchan.num_tiposclasses = sar_ecchan.num_tiposclasses.fillna(sar_ecchan.num_tiposclasses.median())
sar_ecchan.horas_alocadas = sar_ecchan.horas_alocadas.fillna(sar_ecchan.horas_alocadas.median())
sar_ecchan.horassemana_sem_aula = sar_ecchan.horassemana_sem_aula.fillna(sar_ecchan.horassemana_sem_aula.median())

In [145]:
saresp_est = {}
saresp_est['id_escola'] = []
saresp_est['nota_saresp'] = []
saresp_est['id_diretoria'] = []
saresp_est['nr_cluster'] = []
saresp_est['num_alunos'] = []
saresp_est['num_tiposclasses'] = []
saresp_est['horas_alocadas'] = []
saresp_est['horassemana_sem_aula'] = []
for nome, dados in sar_ecchan.groupby('id_escola'):
    saresp_est['id_escola'].append(nome)
    saresp_est['nota_saresp'].append(dados.nota_saresp.mean())
    saresp_est['id_diretoria'].append(dados.CD_DIRETORIA.iloc[0])
    saresp_est['nr_cluster'].append(dados.NR_CLUSTER.iloc[0])
    saresp_est['num_alunos'].append(dados.num_alunos.mean())
    saresp_est['num_tiposclasses'].append(dados.num_tiposclasses.mean())
    saresp_est['horas_alocadas'].append(dados.horas_alocadas.mean())
    saresp_est['horassemana_sem_aula'].append(dados.horassemana_sem_aula.mean())
saresp_est = pd.DataFrame(saresp_est)    

In [148]:
saresp_est.corr()

,id_escola,nota_saresp,nr_cluster,num_alunos,num_tiposclasses,horas_alocadas,horassemana_sem_aula
id_escola,1.000000,-0.182887,-0.001180,0.012331,-0.098849,0.005287,0.002719
nota_saresp,-0.182887,1.000000,0.042086,-0.044213,-0.045944,-0.024975,-0.120687
nr_cluster,-0.001180,0.042086,1.000000,0.104554,-0.032782,0.086577,0.074896
num_alunos,0.012331,-0.044213,0.104554,1.000000,0.018284,0.891379,0.936282
num_tiposclasses,-0.098849,-0.045944,-0.032782,0.018284,1.000000,0.052495,0.084976
horas_alocadas,0.005287,-0.024975,0.086577,0.891379,0.052495,1.000000,0.925547
horassemana_sem_aula,0.002719,-0.120687,0.074896,0.936282,0.084976,0.925547,1.000000


In [151]:
por_diretoria = {}
por_diretoria['diretoria'] = []
por_diretoria['nota_saresp'] = []
for diretoria in set(saresp_est.id_diretoria):
    dir_corte = saresp_est[saresp_est.id_diretoria == diretoria]
    por_diretoria['diretoria'].append(diretoria)
    por_diretoria['nota_saresp'].append(dir_corte.nota_saresp.mean())
por_diretoria = pd.DataFrame(por_diretoria)

In [153]:
por_diretoria.describe()

,nota_saresp
count,93.000000
mean,253.685739
std,13.336674
min,221.823734
25%,244.482588
50%,255.817976
75%,263.440401
max,284.783333


In [155]:
por_diretoria[por_diretoria.nota_saresp == por_diretoria.nota_saresp.max()]

,diretoria,nota_saresp
41,20305,284.783333


In [156]:
por_diretoria[por_diretoria.nota_saresp == por_diretoria.nota_saresp.min()]

,diretoria,nota_saresp
4,10211,221.823734


In [158]:
por_cluster = {}
por_cluster['nr_cluster'] = []
por_cluster['nota_saresp'] = []
for nr_cluster in set(saresp_est.nr_cluster):
    dir_corte = saresp_est[saresp_est.nr_cluster == nr_cluster]
    por_cluster['nr_cluster'].append(nr_cluster)
    por_cluster['nota_saresp'].append(dir_corte.nota_saresp.mean())
por_cluster = pd.DataFrame(por_cluster)

In [179]:
por_cluster

,nr_cluster,nota_saresp
0,0,226.767894
1,1,250.574149
2,2,249.183212
3,3,248.886686
4,4,249.113121
5,5,245.657515
6,6,253.341034
7,7,252.833188
8,8,252.042427
9,9,262.074722


In [160]:
por_cluster[por_cluster.nota_saresp == por_cluster.nota_saresp.max()]

,nr_cluster,nota_saresp
9,9,262.074722


In [161]:
por_cluster[por_cluster.nota_saresp == por_cluster.nota_saresp.min()]

,nr_cluster,nota_saresp
0,0,226.767894


In [177]:
end[end.CD_DIRETORIA == '10211']

,nomedep,depadm,mun,codmun,de,id_escola,CD_DIRETORIA,CD_DIRETORIA_ESTADUAL,CD_DIRETORIA_SUPVS_PROPR,NM_COMPLETO_ESCOLA,CD_UNIDADE,DS_ENDERECO,COMPLEMENTO,NUMERO,CD_ORIGEM_UNIDADE,CD_ORIGEM_ESCOLA,CD_ORIGEM_ENDERECO,LATITUDE,LONGITUDE
70,ESTADUAL - SE,1,SAO PAULO,100,LESTE 3,902724,10211,10211,10211,FERNANDO MAURO PIRES DA ROCHA DEPUTADO,34434,JOSE PINTO,RUA,10,34434,902724,34494,-23.6068877975285,-46.4024754407532
71,ESTADUAL - SE,1,SAO PAULO,100,LESTE 3,904314,10211,10211,10211,GUERRA JUNQUEIRO,6410,FLORIANO ARRESTI,RUA,52,6410,904314,6470,-23.5650320083988,-46.3937535881996
72,ESTADUAL - SE,1,SAO PAULO,100,LESTE 3,923916,10211,10211,10211,MARCOS ANTONIO COSTA PROFESSOR,46944,GODOI BUENO,RUA,19,46944,923916,47004,-23.6306913800994,-46.4431632050018
115,ESTADUAL - SE,1,SAO PAULO,100,LESTE 3,438112,10211,10211,10211,SITIO CONCEICAO,177468,WILSON FERNANDO S.CARVALHO,RUA,35,177468,438112,211868,-23.5781044,-46.391658
116,ESTADUAL - SE,1,SAO PAULO,100,LESTE 3,908368,10211,10211,10211,OSWALDO GAGLIARDI,6438,RUA INACIO MONTEIRO,RUA,3206,6438,908368,6498,-23.5782,-46.3911
117,ESTADUAL - SE,1,SAO PAULO,100,LESTE 3,447663,10211,10211,10211,JARDIM SANTO ANDRE III,191334,MIGUEL FERREIRA DE MELO,RUA,50 A,191334,447663,225734,-23.6302,-46.4435
367,ESTADUAL - SE,1,SAO PAULO,100,LESTE 3,37060,10211,10211,10211,ROCCA DORDALL,37833,RUA DOUTOR RODRIGUES DE ALMEIDA,VILA BUENOS AIRES,143,37833,37060,37893,-23.5536,-46.4182
420,ESTADUAL - SE,1,SAO PAULO,100,LESTE 3,37059,10211,10211,10211,HUMBERTO DANTAS,25373,SERRA DO MAR,RUA,50,25373,37059,25433,-23.5491,-46.4157
614,ESTADUAL - SE,1,SAO PAULO,100,LESTE 3,3268,10211,10211,10211,SEBASTIAO FARIA ZIMBRES PROFESSOR,43529,QUINTA DO PACO,RUA,45,43529,3268,43589,-23.5529045572525,-46.4089329462601
615,ESTADUAL - SE,1,SAO PAULO,100,LESTE 3,43746,10211,10211,10211,FRANCISCO DE ASSIS PIRES CORREA PROFESSOR,12847,RUA SILVIANOPOLIS,ITAQUERA,20,12847,43746,12907,-23.5493820005707,-46.4336901540756


In [163]:
idesp = etl_idesp.etl()

In [174]:
idesp[idesp.id_escola == 915087]

,id_gestao,id_escola,ano_gestao,nota_ano,variacao,anterior_nulo,atual_nulo,unidades_diretoria,nome_diretoria
23650,esc-915087-ano-2014,915087,2014,3.440,39.553753,0,0,31,ITAPEVA
23651,esc-915087-ano-2015,915087,2015,4.725,37.354651,0,0,31,ITAPEVA
23652,esc-915087-ano-2016,915087,2016,6.065,28.359788,0,0,31,ITAPEVA
23653,esc-915087-ano-2017,915087,2017,4.220,-30.420445,0,0,31,ITAPEVA
23654,esc-915087-ano-2018,915087,2018,3.345,-20.734597,0,0,31,ITAPEVA
